In [2]:
import sqlite3

# Connect to the database (creates a new one if it doesn't exist)
conn = sqlite3.connect("app-prod.sqlite")

# Create a cursor object to execute queries
cursor = conn.cursor()

# Execute a query to fetch all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables:", tables)
# Iterate through each table and print its schema
for table in tables:
    table_name = table[0]
    print(f"Schema for table {table_name}:")
    cursor.execute(f"PRAGMA table_info({table_name});")
    schema = cursor.fetchall()
    for column in schema:
        print(column)
    print()


Tables: [('Logs',), ('Users',), ('LibraryLogs',)]
Schema for table Logs:
(0, 'id', 'INTEGER', 1, None, 1)
(1, 'roll_number', 'VARCHAR(50)', 1, None, 0)
(2, 'time', 'DATETIME', 1, None, 0)
(3, 'mode', 'VARCHAR(50)', 1, None, 0)

Schema for table Users:
(0, 'id', 'INTEGER', 1, None, 1)
(1, 'roll_number', 'VARCHAR(50)', 1, None, 0)
(2, 'name', 'VARCHAR(100)', 1, None, 0)
(3, 'designation', 'VARCHAR(50)', 1, None, 0)
(4, 'year', 'INTEGER', 0, None, 0)

Schema for table LibraryLogs:
(0, 'id', 'INTEGER', 1, None, 1)
(1, 'roll_number', 'VARCHAR(50)', 1, None, 0)
(2, 'in_time', 'DATETIME', 1, None, 0)
(3, 'out_time', 'DATETIME', 0, None, 0)
(4, 'time_spent', 'INTEGER', 0, '0', 0)



In [3]:
# -------------------------------
# 1. Modeling Positive Data Using a Gamma Distribution
#    with both MoM and MLE estimation.
# -------------------------------
import numpy as np
import scipy.stats as st
import scipy.optimize as opt
from scipy.special import psi  # digamma
import math
import sqlite3
import pandas as pd
from pandas import DataFrame

# Connect to SQLite database
conn = sqlite3.connect("app-prod.sqlite")
np.random.seed(41)

# Query to get visit_date and visitor_count, sorted by visitor_count
# only for January and February 
query = """
SELECT DATE(in_time) AS visit_date, COUNT(DISTINCT roll_number) AS visitor_count
FROM LibraryLogs
WHERE strftime('%m', in_time) IN ('01', '02')
GROUP BY visit_date
"""

# Execute the query and load the result into a DataFrame
cursor = conn.cursor()
df_visitors_sorted = DataFrame(cursor.execute(query).fetchall(), columns=["visit_date", "visitor_count"])

# Select 30 random data points from the DataFrame
random_data_points = df_visitors_sorted.sample(n=30, random_state=41)["visitor_count"].to_numpy()

# --- (a) Method of Moments (MoM) Estimates ---
# Calculate sample mean and E[X^2]
sample_mean = np.mean(random_data_points)
sample_ex2 = np.mean(random_data_points**2)  # E[X^2]

# Method of Moments (MoM) estimates
a_mom = sample_mean**2 / (sample_ex2 - sample_mean**2)
b_mom = sample_mean / (sample_ex2 - sample_mean**2) 

print("Method of Moments Estimates:")
print("1st sample moment, E[X]:", f"{sample_mean:.2f}")
print("2nd sample moment, E[X^2]:", f"{sample_ex2:.2f}")
print("Estimated shape (a):", f"{a_mom:.4f}")
print("Estimated rate (b):", f"{b_mom:.4f}")

# --- (b) Maximum Likelihood Estimation (MLE) ---
# Define the log-likelihood function for the Gamma distribution
def log_likelihood(params):
    a, b = params
    return np.sum((a - 1) * np.log(random_data_points) - random_data_points / b - a * np.log(b) - math.lgamma(a))

# Initial guesses for a and b (using MoM estimates)
initial_guess = [a_mom, b_mom]

# Perform optimization to maximize the log-likelihood
result = opt.minimize(
    fun=lambda params: -log_likelihood(params),  # Minimize the negative log-likelihood
    x0=initial_guess,
    bounds=[(1e-6, None), (1e-6, None)]  # Ensure a and b are positive
)

# Extract the MLE estimates for a and b
a_mle, b_mle = result.x

print("\nMaximum Likelihood Estimates:")
print("Estimated shape (a):", f"{a_mle:.4f}")
print("Estimated rate (b):", f"{1/b_mle:.4f}")


Method of Moments Estimates:
1st sample moment, E[X]: 455.90
2nd sample moment, E[X^2]: 223161.63
Estimated shape (a): 13.5697
Estimated rate (b): 0.0298

Maximum Likelihood Estimates:
Estimated shape (a): 11.6488
Estimated rate (b): 0.0256


In [4]:
# -------------------------------
# 2. Confidence Interval for Variance under a Normal Model
# -------------------------------

import numpy as np
import scipy.stats as st
import scipy.optimize as opt
from scipy.special import psi  # digamma
import math
import sqlite3
import pandas as pd
from pandas import DataFrame

# Connect to SQLite database
conn = sqlite3.connect("app-prod.sqlite")
np.random.seed(41)

# Query to get visit_date and visitor_count, sorted by visitor_count
# only for January and February 
query = """
SELECT DATE(in_time) AS visit_date, COUNT(DISTINCT roll_number) AS visitor_count
FROM LibraryLogs
WHERE strftime('%m', in_time) IN ('01', '02')
GROUP BY visit_date
"""

# Execute the query and load the result into a DataFrame
df_visitors_sorted = DataFrame(cursor.execute(query).fetchall(), columns=["visit_date", "visitor_count"])

# Select 30 random data points from the DataFrame
random_data_points = df_visitors_sorted.sample(n=30, random_state=41)["visitor_count"].to_numpy()

# Calculate sample variance
sample_variance = np.var(random_data_points, ddof=1)  # unbiased sample variance
n = len(random_data_points)  # sample size
alpha = 0.05  # significance level for 95% confidence interval

# Critical values for chi-square distribution
chi2_lower = st.chi2.ppf(alpha / 2, df=n - 1)
chi2_upper = st.chi2.ppf(1 - alpha / 2, df=n - 1)

# Confidence interval for variance
ci_variance_lower = (n - 1) * sample_variance / chi2_upper
ci_variance_upper = (n - 1) * sample_variance / chi2_lower

print(f"95% Confidence Interval for Variance: [{ci_variance_lower:.2f}, {ci_variance_upper:.2f}]")


95% Confidence Interval for Variance: [10049.91, 28634.80]


In [5]:
# -------------------------------
# 3. Confidence Interval for the Difference in Means of Two Independent Normal Populations
# -------------------------------
# Query to find the number of visitors per day with roll number "23BTECH" in the month of November
query_sem_03 = """
SELECT DATE(in_time) AS visit_date, COUNT(DISTINCT roll_number) AS visitor_count
FROM LibraryLogs
WHERE strftime('%m', in_time) = '11' AND roll_number LIKE '%23BTECH%'
GROUP BY visit_date
"""

# Execute the query and load the result into a DataFrame
df_sem_03 = pd.DataFrame(cursor.execute(query_sem_03).fetchall(), columns=["visit_date", "visitor_count"])

# Query to find the number of visitors per day with roll number "23BTECH" in the month of January
query_sem_04 = """
SELECT DATE(in_time) AS visit_date, COUNT(DISTINCT roll_number) AS visitor_count
FROM LibraryLogs
WHERE strftime('%m', in_time) = '01' AND roll_number LIKE '%23BTECH%'
GROUP BY visit_date
"""

# Execute the query and load the result into a DataFrame
df_sem_04 = pd.DataFrame(cursor.execute(query_sem_04).fetchall(), columns=["visit_date", "visitor_count"])

# Convert data into numpy arrays
visitors_sem_03 = df_sem_03["visitor_count"].to_numpy()
visitors_sem_04 = df_sem_04["visitor_count"].to_numpy()

# Calculate sample mean and variance for both datasets
mean_sem_03 = np.mean(visitors_sem_03)
variance_sem_03 = np.var(visitors_sem_03, ddof=1)

mean_sem_04 = np.mean(visitors_sem_04)
variance_sem_04 = np.var(visitors_sem_04, ddof=1)

n_sem_03 = len(visitors_sem_03)
n_sem_04 = len(visitors_sem_04)
# Print the results
print("Number of points in Semester 03 dataset:", n_sem_03)
print("Number of points in Semester 04 dataset:", n_sem_04)
print("\nSemester 03 Visitors - Mean:", f"{mean_sem_03:.3f}", ", Variance:", f"{variance_sem_03:.3f}")
print("Semester 04 Visitors - Mean:", f"{mean_sem_04:.3f}", ", Variance:", f"{variance_sem_04:.3f}")

# Pooled variance
pooled_variance = ((n_sem_03 - 1) * variance_sem_03 + (n_sem_04 - 1) * variance_sem_04) / (n_sem_03 + n_sem_04 - 2)

# Standard error for the difference in means
se_diff_equal_var = np.sqrt(pooled_variance * (1 / n_sem_03 + 1 / n_sem_04))

# Critical t value for 95% confidence interval
t_crit_equal_var = st.t.ppf(0.975, df=n_sem_03 + n_sem_04 - 2)

# Confidence interval for the difference in means
ci_diff_lower_equal_var = mean_sem_03 - mean_sem_04 - t_crit_equal_var * se_diff_equal_var
ci_diff_upper_equal_var = mean_sem_03 - mean_sem_04 + t_crit_equal_var * se_diff_equal_var

print(f"\nConfidence Interval for Difference of Means (Equal Variance Assumption): [{ci_diff_lower_equal_var:.2f}, {ci_diff_upper_equal_var:.2f}]")


Number of points in Semester 03 dataset: 29
Number of points in Semester 04 dataset: 31

Semester 03 Visitors - Mean: 71.862 , Variance: 886.980
Semester 04 Visitors - Mean: 70.452 , Variance: 844.656

Confidence Interval for Difference of Means (Equal Variance Assumption): [-13.80, 16.62]


In [16]:
# -------------------------------
# 4. Hypothesis Testing for a Bernoulli-Distributed (Binary) Response
# -------------------------------
# Query to check if roll number "AI23BTECH11022" visited the library each day in January and February
query_ai23btech = """
WITH all_dates AS (
    SELECT DATE(in_time) AS visit_date
    FROM LibraryLogs
    WHERE strftime('%m', in_time) IN ('01', '02')
    GROUP BY visit_date
)
SELECT all_dates.visit_date, 
       CASE WHEN COUNT(LibraryLogs.roll_number) > 0 THEN 1 ELSE 0 END AS visited
FROM all_dates
LEFT JOIN LibraryLogs 
ON all_dates.visit_date = DATE(LibraryLogs.in_time) 
   AND LibraryLogs.roll_number = 'AI23BTECH11022'
GROUP BY all_dates.visit_date
ORDER BY all_dates.visit_date;
"""

# Execute the query and load the result into a DataFrame
df_ai23btech = pd.DataFrame(cursor.execute(query_ai23btech).fetchall(), columns=["visit_date", "visited"])

visited_array = df_ai23btech["visited"].to_numpy()

# Perform hypothesis testing for H0: p <= 0.5 vs H1: p > 0.5
import pandas as pd
from scipy.stats import binom

# Extract the binary array of visits
n = len(visited_array)
X_obs = visited_array.sum()  # Observed number of visits

# Parameters
p0 = 0.5
alpha = 0.05

# Find k* using the survival function
k_star = None
for k in range(n + 1):
    # Compute P(X >= k) under p0 = 0.5
    prob = binom.sf(k - 1, n, p0)
    if prob <= alpha:
        k_star = k
        break

# Hypothesis test conclusion
if k_star is not None and X_obs >= k_star:
    conclusion = f"Reject H₀. Observed visits ({X_obs}) ≥ critical value ({k_star})."
else:
    conclusion = f"Accept H₀. Observed visits ({X_obs}) < critical value ({k_star})."

print(f"Sample size (n): {n}")
print(f"Observed visits (X_obs): {X_obs}")
print(f"Critical value (k*): {k_star}")
print(conclusion)


Sample size (n): 58
Observed visits (X_obs): 23
Critical value (k*): 36
Accept H₀. Observed visits (23) < critical value (36).
